<a href="https://colab.research.google.com/github/durham-abric/Comp551_Final_Proj/blob/master/base_lm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import tensorflow as tf
from tensorflow import concat
from tensorflow.layers import Conv1D
from tensorflow.layers import Dense
from tensorflow.layers import Dropout
from tensorflow.layers import Flatten
from tensorflow.contrib import rnn
from tensorflow.contrib.rnn import InputProjectionWrapper
from tensorflow.nn import relu
from tensorflow.nn import softmax

In [0]:
def weights_init(shape):
    return tf.Variable(tf.truncated_normal(shape=shape, stddev=0.05))
  
def bias_init(shape):
    return tf.Variable(tf.zeros(shape=shape))

In [0]:
def embedding_layer(vocab_size, embeding_size, inputs):
  
    word_embedings = tf.Variable(tf.random_uniform([vocab_size, embeding_size]))
    embed = tf.nn.embedding_lookup(word_embedings, inputs)
    embed_expended = tf.expand_dims(embed, -1) #expend dims to 4d for conv layer
    return embed_expended

In [0]:
def define_inputs(batch_size, sequence_len):
  
    inputs = tf.placeholder(tf.int32, [batch_size, sequence_len], name='inputs')
    targets = tf.placeholder(tf.float32, [batch_size, 1], name='target')
    keep_probs = tf.placeholder(tf.float32, name='keep_probs')
    
    return inputs, targets, keep_probs

In [0]:
def conv_1d(input, kernel_size, number_of_filters, strides=(1, 1), activation=tf.nn.relu, max_pool=True):
    weights = weights_init([kernel_size, kernel_size, 1, number_of_filters])
    bias = bias_init([number_of_filters])
  
    layer = tf.nn.conv2d(input, filter=weights, strides=[1, strides[0], strides[1], 1], padding='SAME')
    
    if activation != None:
        layer = activation(layer)
    
    if max_pool:
        layer = tf.nn.max_pool(layer, ksize=[1, 2, 2 ,1], strides=[1, 2, 2, 1], padding='SAME')
    
    return layer

In [0]:
def dense(input, in_size, out_size, dropout=0.25, activation=tf.nn.relu):
    weights = weights_init([in_size, out_size])
    bias = bias_init([out_size])
    
    layer = tf.matmul(input, weights) + bias
    
    if activation != None:
        layer = activation(layer)
    
    if dropout:
        layer = tf.nn.dropout(layer, dropout)
        
    return layer

In [0]:
def lstm_layer(lstm_size, number_of_layers, batch_size, dropout_rate):
  
    def cell(size, dropout_rate=None):
        layer = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        
        return tf.contrib.rnn.DropoutWrapper(layer, output_keep_prob=dropout_rate)
            
    cell = tf.contrib.rnn.MultiRNNCell([cell(lstm_size, dropout_rate) for _ in range(number_of_layers)])
    
    init_state = cell.zero_state(batch_size, tf.float32)
    return cell, init_state

In [0]:
class LanguageModel(object):
  
  def __init__(self, learning_rate=0.02, dropout_rate=0.25, batch_size=128, project_dim = 1024, seq_len=50, vocab_size=1024, embed_size=300,
               number_of_lstm_layers=2, lstm_units=128):
    
    tf.reset_default_graph()
    self.inputs, self.targets, self.keep_probs = define_inputs(batch_size, seq_len)

    word_embeddings = tf.get_variable('word_embeddings', [vocab_size, embed_size])
    embedded_word_ids = tf.nn.embedding_lookup(word_embeddings, self.inputs)

    
#     embeded_words = 

    # Neural network
    # 8 conv-1d in parallel concatenated
    conv_block_1 = tf.layers.conv1d(embedded_word_ids, 32, 1)
#     conv_block_2 = tf.layers.conv1d(embedded_word_ids, 32, 2)
#     conv_block_3 = tf.layers.conv1d(embedded_word_ids, 64, 3)
#     conv_block_4 = tf.layers.conv1d(embedded_word_ids, 128, 4)
#     conv_block_5 = tf.layers.conv1d(embedded_word_ids, 256, 5)
#     conv_block_6 = tf.layers.conv1d(embedded_word_ids, 512, 6)
#     conv_block_7 = tf.layers.conv1d(embedded_word_ids, 1028, 7)
#     conv_block_8 = tf.layers.conv1d(embedded_word_ids, 2048, 7)
    conv_layer = [conv_block_1]#, conv_block_2, conv_block_3,
#                   conv_block_4, conv_block_5, conv_block_6,
#                   conv_block_7, conv_block_8]
    
    concatenated_conv = tf.concat(conv_layer, 0)
    
    # Feed Forward Layers
    logits_1 = tf.layers.dense(concatenated_conv, 4096)
    logits_2 = tf.layers.dense(logits_1, 4096)
    
#     # Projection
#     ## TODO
    
#     # LSTM Layers
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_units)
    lstm_state = lstm.zero_state(batch_size, dtype=tf.float32)
    logits_3, lstm_state = lstm_layer(logits_2, lstm_state, batch_size, dropout_rate)
    logits_4, lstm_state = lstm_layer(logits_3, lstm_state, batch_size, dropout_rate)

    
#     # Predictions softmax
    logits_5 = tf.layers.dense(logits_4, 800000)

    
    

In [73]:
model = LanguageModel()

TypeError: ignored